### Finding files and data using Data Connect

#### Learning Objectives
Workshop attendees will learn how use the GA4GH Data Connect Service.  

What will participants do as part of the exercise?

 - Understanding how to query data via Data Connect
 - Use Data Connect to find files that can be accessed via DRS
 - Learn how to obtain and use data descriptions (schema)
 - Discover the meaning of codes used in data
 

 #### Icons in this Guide

 🖐 A hands-on section where you will code something or interact with the server
 
### Query files
The approach taken below is using mapping available through subject and specimen data available through the Data Connect API. 

Queries are submitted as SQL queries to one or more tables on the Data Connect server.

As with other examples, first we set up a client to use the API. The server at DNAStack is used in the following examples.

#### Step 1: Set up a Data Connect Client and run a predefined query 

In [1]:
from fasp.search import DataConnectClient
searchClient = DataConnectClient('https://data.publisher.dnastack.com/data-connect/')

In [2]:
from fasp.search import DataConnectClient

query = '''SELECT f.sample_name, drs_id bam_drs_id, acc
FROM collections.public_datasets.onek_genomes_ssd_drs s 
join collections.public_datasets.onek_genomes_sra_drs_files f on f.sample_name = s.su_submitter_id 
where filetype = 'bam' and mapped = 'mapped' 
and sequencing_type ='exome' and  population = 'JPT' '''

resultRows = searchClient.run_query(query, returnType='dataframe')
resultRows

Retrieving the query
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________


,sample_name,bam_drs_id,acc
0,NA18945,9327fb44eb81b49a41e38c8d86eb3b3a,SRR1601115
1,NA18942,5c6244a1c2f499b356a1198b808383d9,SRR1604445
2,NA18948,fb1cfb04d3ef99d07c21f9dbf87ccc68,SRR1601121
3,NA18949,59dafcd573c26f7f481bcc2b72bda510,SRR1601123
4,NA18947,4673e7afdcb89edf37fd974e1bf40e15,SRR1601119
...,...,...,...
99,NA19080,6f9f1fc52166530ed0568d61451b032f,SRR1598080
100,NA19074,0805baa0849485a2a63ea41429b9b37c,SRR1604135
101,NA19091,13ab0c2657909b758428821e85e5c91f,SRR1603948
102,NA19086,1fe317586a8b8c2a8ca6dadebb9a0b4b,SRR1601181


#### Step 2: Run a second query to find bam files from members of a given family

In [3]:
family_query = '''SELECT f.sample_name, relationship, drs_id bam_drs_id, acc
FROM collections.public_datasets.onek_genomes_thousand_genomes_meta s 
join collections.public_datasets.onek_genomes_sra_drs_files f on f.sample_name = s.sample 
where filetype = 'bam' and mapped = 'mapped' 
and sequencing_type ='exome' and  family_id = '1447' '''

family_results = searchClient.run_query(family_query, returnType='dataframe')
family_results

Retrieving the query
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________


,sample_name,relationship,bam_drs_id,acc
0,NA12760,pat grandfather,a4ee62278644957c2f7ed39202184fdc,SRR1597880
1,NA12762,mat grandfather,d3b77ffe47b452f92c061309ebab587d,SRR1601869
2,NA12761,pat grandmother,57864dd794f647992c188dbea53a13df,SRR1601867
3,NA12763,mat grandmother,988a24e166d8e103406cd62f7857b8b6,SRR1601870


### List table details


#### Step 3:
We can list the available tables available in this set as follows

In [4]:
table_list = searchClient.list_catalog('collections')  # This will list all accessible tables.

Retrieving the table list
____Page1_______________
thousand_genomes.onek_genomes.bdc_1000genomes
thousand_genomes.onek_genomes.onek_drs
thousand_genomes.onek_genomes.onek_recal_variants_drs
thousand_genomes.onek_genomes.sra_drs_files
thousand_genomes.onek_genomes.sra_onek_drs
thousand_genomes.onek_genomes.ssd_drs
thousand_genomes.onek_genomes.ssd_drs_copy
thousand_genomes.onek_genomes.thousand_genomes_meta


#### Step 4: List schema for sra_drs_files table
The following cells can be run to list the columns for the other tables used in the queries above.

In [5]:
schema1 = searchClient.list_table_info('collections.public_datasets.onek_genomes_sra_drs_files', verbose=True)

_Schema for tablethousand_genomes.onek_genomes.sra_drs_files_
{
   "name": "thousand_genomes.onek_genomes.sra_drs_files",
   "description": "Thousand genomes Sequence Read Archive file data",
   "data_model": {
      "$id": "TG_sra_t1",
      "description": "Thousand genomes Sequence Read Archive file data",
      "properties": {
         "acc": {
            "type": "string",
            "$id": "TG_sra_v1",
            "description": "SRA run accession no"
         },
         "drs_id": {
            "type": "string",
            "$id": "TG_sra_v3",
            "description": "local DRS id unique within the DRS service"
         },
         "mapped": {
            "type": "encoded value",
            "$id": "TG_sra_v6",
            "oneOf": [
               {
                  "const": "mapped",
                  "title": "mapped to reference"
               },
               {
                  "const": "unmapped",
                  "title": "unmapped to reference"
               }
 

In [6]:
schema2 = searchClient.list_table_info('collections.public_datasets.onek_genomes_ssd_drs', verbose=True)

_Schema for tablethousand_genomes.onek_genomes.ssd_drs_
{
   "name": "thousand_genomes.onek_genomes.ssd_drs",
   "description": "Thousand genomes sample metadata",
   "data_model": {
      "$id": "TG_sra_t2",
      "description": "Thousand genomes sample metadata",
      "properties": {
         "md5sum": {
            "type": "number",
            "$id": "TG_sra_v21",
            "description": "MD5 checksum for file"
         },
         "data_type": {
            "type": "encoded value",
            "$id": "TG_sra_v22",
            "oneOf": [
               {
                  "const": "Aligned_20_Reads",
                  "title": "Aligned_20_Reads"
               }
            ],
            "description": "Type of data contained in file"
         },
         "file_name": {
            "type": "string",
            "$id": "TG_sra_v19",
            "description": "Name of file"
         },
         "file_size": {
            "type": "number",
            "$id": "TG_sra_v20",
      

#### Step 6: Search for a different population group 
🖐 Using the information above about the tables, modify the query to use
a) a population code that represents Gujarati Indians living in a city in Texas.
b) bam files for reads that have not been mapped to a reference genome.

So you don't have to modify the sql query itself you can add the values you identified to the variables in the next cell of this notebook.

In [7]:
population_code = 'GIH'
mapping_type = 'unmapped'

In [8]:
query = f'''SELECT f.sample_name, drs_id bam_drs_id, acc, filename
FROM collections.public_datasets.onek_genomes_ssd_drs s 
join collections.public_datasets.onek_genomes_sra_drs_files f on f.sample_name = s.su_submitter_id 

where filetype = 'bam' and mapped = '{mapping_type}' 
and sequencing_type ='exome' and  population = '{population_code}'
'''

resultRows = searchClient.run_query(query, returnType='dataframe')
resultRows

Retrieving the query
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________


,sample_name,bam_drs_id,acc,filename
0,NA20853,1f0a7f9e10ebf38d45bbb08cffe69851,SRR1598480,NA20853.unmapped.ILLUMINA.bwa.GIH.exome.201205...
1,NA20852,b55c589bfcabc81355b2fdbb2f81ff37,SRR1598478,NA20852.unmapped.ILLUMINA.bwa.GIH.exome.201205...
2,NA20894,dcdd7fcafa21f5d69a0ab650114faf88,SRR1598538,NA20894.unmapped.ILLUMINA.bwa.GIH.exome.201205...
3,NA20895,7c45236b410cef0bc85e6052f810ef69,SRR1598540,NA20895.unmapped.ILLUMINA.bwa.GIH.exome.201205...
4,NA20899,2a8edfc0ffaa964348fbbc0c15d7231c,SRR1598548,NA20899.unmapped.ILLUMINA.bwa.GIH.exome.201205...
...,...,...,...,...
98,NA21114,bc21dbc917062f4c6e79e08f00bc2bcf,SRR1598598,NA21114.unmapped.ILLUMINA.bwa.GIH.exome.201205...
99,NA21107,8bd32cc867deeab6c8d5e9fd4eca8ff6,SRR1600518,NA21107.unmapped.ILLUMINA.bwa.GIH.exome.201304...
100,NA21111,928ccbddac73668de5b61d4bb4d0df4e,SRR1600526,NA21111.unmapped.ILLUMINA.bwa.GIH.exome.201304...
101,NA21108,fd0bdfa174cf6e721c4b3921bb840cfc,SRR1602948,NA21108.unmapped.ILLUMINA.bwa.GIH.exome.201304...


### Important note
Looking up a data dictionary to discover codes in this way is not what we would typically expect a user to do. Our aim today is to focus on the API and what it is capable of and what it can enable.

Given the information the data schema provide about the data it is possible for developers to create interfaces in their systems which allow new datasources to be integrated as they appear.

In another notebook (next). We'll look at an example of how a more user friendly user interface can be provided using the information that the the API provides.

#### Step 7 - Combine with DRS Server

The following shows how the SRA DRS server we used in workbook 2-1 can be used to determine where the files we discovered can be obtained from. 

🖐 Using the results from one of the queries that you ran above take a DRS id from the query results and use it in the following calls to the NCBI DRS server.

In [9]:
from fasp.loc import DRSClient

drsClient = DRSClient('https://locate.be-md.ncbi.nlm.nih.gov', public=True, debug=True)
test_id = '1f0a7f9e10ebf38d45bbb08cffe69851'
objInfo = drsClient.get_object(test_id)
objInfo

https://locate.be-md.ncbi.nlm.nih.gov/ga4gh/drs/v1/objects/1f0a7f9e10ebf38d45bbb08cffe69851


{'access_methods': [{'access_id': '6d8f0e53659a16fba26e9abc4b40d3930929c448bd54f76398dac8788c90bf56',
   'region': 'gs.US',
   'type': 'https'},
  {'access_id': '01894b008317005f919216578ffd8ef095c30d4541489f3db35e358bb74c7450',
   'type': 'https'},
  {'access_id': '5910ee14f991a7db0fa75bd24a4c5e169c206c09d9787e0fbed6a5ae4765b9a9',
   'region': 's3.us-east-1',
   'type': 'https'}],
 'checksums': [{'checksum': '1f0a7f9e10ebf38d45bbb08cffe69851',
   'type': 'md5'}],
 'created_time': '2012-11-18T13:03:48Z',
 'id': '1f0a7f9e10ebf38d45bbb08cffe69851',
 'name': 'NA20853.unmapped.ILLUMINA.bwa.GIH.exome.20120522.bam',
 'self_url': 'drs://locate.be-md.ncbi.nlm.nih.gov/1f0a7f9e10ebf38d45bbb08cffe69851',
 'size': 114284941}

A second DRS call can be used to obtain a url to access the file from one of the above locations.

In [10]:
access_id = objInfo['access_methods'][0]['access_id']
print('access_id:{}'.format(access_id))
url = drsClient.get_access_url(test_id, access_id=access_id)
print('url:{}'.format(url))

access_id:6d8f0e53659a16fba26e9abc4b40d3930929c448bd54f76398dac8788c90bf56
https://locate.be-md.ncbi.nlm.nih.gov/ga4gh/drs/v1/objects/1f0a7f9e10ebf38d45bbb08cffe69851/access/6d8f0e53659a16fba26e9abc4b40d3930929c448bd54f76398dac8788c90bf56
<Response [200]>
url:https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA20853/exome_alignment/NA20853.unmapped.ILLUMINA.bwa.GIH.exome.20120522.bam
